# DeleteInstance

<a href="#" onclick="history.back()">&lt;--- Back</a>

For some CIM classes, it makes no sense to support deletion of their CIM instances. For some others, deletion of its CIM instances and consequently deletion of the represented managed resources is defined. Such a deletion semantics is typically not defined in the class definitions in the CIM Schema, but in their usage definitions in WBEM Management Profiles (see DMTF standard DSP1001). Often, management profiles that define a semantics for the deletion of managed resources, leave that optional for an implementation to support. The implementation for a CIM class in the WBEM server (aka CIM provider) thus may or may not support deletion of its instances and the represented managed resources.

The [`DeleteInstance()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.DeleteInstance) method takes a [`pywbem.CIMInstanceName`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstanceName) object and deletes the referenced CIM instance and the represented managed resource, or rejects the operation if deletion is not supported.

In [ ]:
from __future__ import print_function
import sys
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/cimv2'
server = 'http://localhost'

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    cs_paths = conn.EnumerateInstanceNames(classname, namespace)
except pywbem.Error as exc:
    print('EnumerateInstanceNames failed: %s: %s' % (exc.__class__.__name__, exc))
    sys.exit(1)

if len(cs_paths) > 0:
    cs_path = cs_paths[0]
    print('Deleting instance at: %s' % cs_path)
    try:
        conn.DeleteInstance(cs_path)
    except pywbem.Error as exc:
        if isinstance(exc, pywbem.CIMError) and \
           exc.status_code == pywbem.CIM_ERR_NOT_SUPPORTED:
            print('Deletion not supported for instance: %s' % cs_path)
        else:
            print('DeleteInstance failed: %s: %s' % (exc.__class__.__name__, exc))
            sys.exit(1)

This example has a somewhat more elaborated failure message that includes the type of exception that happened.

This example also shows how specific CIM errors can be detected: If deletion of the CIM instance and the corresponding managed resource is not supported, this example code accepts that and does not error out. All other errors, including other CIM errors, cause an error exit.

As it turns out, the CIM provider for class "CIM_ComputerSystem" in our WBEM server does not support the deletion of the system it runs on, so the WBEM server returns a failure with CIM status CIM_ERR_NOT_SUPPORTED.

PyWBEM maps CIM operation failures to the Python exception [`pywbem.CIMError`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMError), and raises that in this case. The CIM status code is available as a numeric value in the `status_code` attribute of the exception object. See [CIM status codes](https://pywbem.readthedocs.io/en/latest/client.html#cim-status-codes) for a definition of the CIM status code values.

<a href="#" onclick="history.back()">&lt;--- Back</a>